In [13]:
import json
from urllib import request, parse
import random
import uuid
import urllib.request
import urllib.parse
import io
import os
import time
from PIL.PngImagePlugin import PngInfo
import hashlib


## import json file

In [14]:
promptJson = 'roop_api.json'

with open(promptJson) as f:
    prompt = json.load(f)

print(str(prompt))

{'12': {'inputs': {'image': '020_0.png', 'choose file to upload': 'image'}, 'class_type': 'LoadImage'}, '34': {'inputs': {'swap_model': 'inswapper_128.onnx', 'faces_index': '0', 'reference_faces_index': '0', 'face_order': 'left to right', 'reverse_order': False, 'reference_order': 'left to right', 'reverse_reference_order': False, 'console_logging_level': 0, 'image': ['12', 0], 'reference_image': ['35', 0]}, 'class_type': 'RoopImproved'}, '35': {'inputs': {'image': '828bbd3b93befac87c8ae2a61dd552f0.jpeg', 'choose file to upload': 'image'}, 'class_type': 'LoadImage'}, '39': {'inputs': {'images': ['34', 0]}, 'class_type': 'PreviewImage'}}


In [15]:
server_address = "127.0.0.1:8188"
client_id = str(uuid.uuid4())


In [16]:
def changeImageSize(width, height):
    # 最小的宽高为 896
    limited = 896
    minSize = min(width, height)

    # 缩放宽高，使得最窄的边等于limited
    if minSize > limited:
        scale = minSize / limited
        widthScale = int(width / scale)
        heightScale = int(height / scale)
    else:
        scale = limited / minSize
        widthScale = int(width * scale)
        heightScale = int(height * scale)

    # 将两个值都除以64，然后取整,也就是说边长必须是64的倍数
    widthScale = int(widthScale/64) * 64
    heightScale = int(heightScale/64) * 64

    return widthScale, heightScale

def checkImageExsit(imageDir, imageName):
    for file in os.listdir(imageDir):
        if file.startswith(imageName):
            return True
    return False


def queue_prompt(prompt):
    p = {"prompt": prompt, "client_id": client_id}
    data = json.dumps(p).encode('utf-8')
    req =  urllib.request.Request("http://{}/prompt".format(server_address), data=data)
    return json.loads(urllib.request.urlopen(req).read())

def get_history(prompt_id):
    with urllib.request.urlopen("http://{}/history/{}".format(server_address, prompt_id)) as response:
        return json.loads(response.read())

def get_image(filename, subfolder, folder_type):
    data = {"filename": filename, "subfolder": subfolder, "type": folder_type}
    url_values = urllib.parse.urlencode(data)
    with urllib.request.urlopen("http://{}/view?{}".format(server_address, url_values)) as response:
        return response.read()

def handle_whitespace(string: str):
    return string.strip().replace("\n", " ").replace("\r", " ").replace("\t", " ")

def parse_name(ckpt_name):
    path = ckpt_name
    filename = path.split("/")[-1]
    filename = filename.split(".")[:-1]
    filename = ".".join(filename)
    return filename


def calculate_sha256(file_path):
    sha256_hash = hashlib.sha256()

    with open(file_path, "rb") as f:
        # Read the file in chunks to avoid loading the entire file into memory
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)

    return sha256_hash.hexdigest()

def get_images(ws, prompt):
    prompt_id = queue_prompt(prompt)['prompt_id']
    output_images = {}
    while True:
        out = ws.recv()
        if isinstance(out, str):
            message = json.loads(out)
            if message['type'] == 'executing':
                data = message['data']
                if data['node'] is None and data['prompt_id'] == prompt_id:
                    break #Execution is done
        else:
            continue #previews are binary data

    history = get_history(prompt_id)[prompt_id]
    for o in history['outputs']:
        for node_id in history['outputs']:
            node_output = history['outputs'][node_id]
            if 'images' in node_output:
                images_output = []
                for image in node_output['images']:
                    image_data = get_image(image['filename'], image['subfolder'], image['type'])
                    images_output.append(image_data)
                output_images[node_id] = images_output

    return output_images
    
def saveImages(result, imageDir,imageName,comment,previewImage=0): 
    metadata = PngInfo()
    metadata.add_text("parameters", comment)  
    k = 0 
    for node_id in result:    
        if k < previewImage:
            k += 1
            continue    
        for image_data in result[node_id]:            
            image = Image.open(io.BytesIO(image_data))
            imagePath = imageDir + "/" + imageName + f"_{k}" + ".png"
            image.save(imagePath,pnginfo=metadata) 
            k += 1
    

In [17]:
# 子文件夹
subDir = "BoA"

# 原图文件夹
originalDir = f"f:/Poledriver/Paid/{subDir}"
# 设置图片保存路径
imageDir = f"f:/Poledriver/roop/{subDir}"
# 如果文件夹不存在，创建文件夹
if not os.path.exists(imageDir):
    os.mkdir(imageDir)
    
# 这里是要替换的人脸图片
faceReferenceImagePath = "f:/Poledriver/kaneko_face.jpg"

In [18]:

from PIL import Image
from sdparsers import ParserManager
import websocket

# 这些参数是乱写的
positive = ''
negative = ''
steps = 35
sampler_name = 'dpmpp_2m_sde'
scheduler = 'karras'
cfg = 7
seed = random.randint(0, 1000000)

batch_size = 1
# 遍历原图文件夹里所有的图片
for originalImag in os.listdir(originalDir):
        # 如果不是jpg或png文件，跳过
        if not (originalImag.endswith(".jpg") or originalImag.endswith(".png") or originalImag.endswith(".jpeg")):
            continue
        
        originalImagePath = originalDir + "/" + originalImag
        # 如果文件名包含_H或者_B，跳过
        if originalImag.find("_H") != -1 or originalImag.find("_B") != -1:
            print(f"{originalImag}包含_H或者_B,判定为mask文件，跳过")
            continue
        
        # 读取原图，以及其大小
        with Image.open(originalImagePath) as originalImage:
            originalImageWidth = originalImage.size[0]
            originalImageHeight = originalImage.size[1]
            parser_manager = ParserManager()
            prompt_data = parser_manager.parse(originalImage)
            # 读取原图的提示词，如果解析失败，使用默认提示词
            try:
                positive = prompt_data.prompts[0][0].value  
                negative = prompt_data.prompts[0][1].value
            except:
                print(f"原图片{originalImag}解析信息失败,使用默认提示词")
        
        # 读取原图片名字，去掉后缀.png
        originalImagName = originalImag.split(".")[0]
        # 检测目标文件夹里是否已经存在该图片，如果存在，跳过
        if checkImageExsit(imageDir, originalImagName):
            print(f"{originalImagName}已经存在，跳过")
            continue 
        
        # roop只需要两个参数，就是原图片和脸部参考图片
        prompt['35']['inputs']['image'] = faceReferenceImagePath
        prompt['12']['inputs']['image'] = originalImagePath
        
        # 模型名字也是随便写的，roop生成过程并不需要用到模型
        modelname = "pandorasBoxNSFW_v1PussBoots.safetensors"
        basemodelname = parse_name(modelname)        
        ckpt_path = "e:/sdwebui/webui/models/Stable-diffusion/" + modelname
        modelhash = calculate_sha256(ckpt_path)[:10]
        
        comment = f"{handle_whitespace(positive)}\nNegative prompt: {handle_whitespace(negative)}\nSteps: {steps}, Sampler: {sampler_name}{f'_{scheduler}' if scheduler != 'normal' else ''}, CFG Scale: {cfg}, Seed: {seed}, Size: {originalImageWidth}x{originalImageHeight}, Model hash: {modelhash}, Model: {basemodelname}, Version: ComfyUI"        
        
        print(f"生成图片的信息是：{comment}")
        
        # 连接api，生成并保存图片
        start_time = time.time()
        ws = websocket.WebSocket()
        ws.connect("ws://{}/ws?clientId={}".format(server_address, client_id))
        images = get_images(ws, prompt)   
        saveImages(images, imageDir, originalImagName, comment=comment,previewImage=0)     
        end_time = time.time()
        print(f"耗时{end_time-start_time}秒")
        

原图片018.jpg解析信息失败,使用默认提示词
生成图片的信息是：
Negative prompt: 
Steps: 35, Sampler: dpmpp_2m_sde_karras, CFG Scale: 7, Seed: 652753, Size: 1280x1920, Model hash: a3cd261baa, Model: pandorasBoxNSFW_v1PussBoots, Version: ComfyUI
耗时6.831702470779419秒
原图片019.jpg解析信息失败,使用默认提示词
生成图片的信息是：
Negative prompt: 
Steps: 35, Sampler: dpmpp_2m_sde_karras, CFG Scale: 7, Seed: 652753, Size: 1280x1920, Model hash: a3cd261baa, Model: pandorasBoxNSFW_v1PussBoots, Version: ComfyUI
耗时6.421063423156738秒
原图片020.jpg解析信息失败,使用默认提示词
生成图片的信息是：
Negative prompt: 
Steps: 35, Sampler: dpmpp_2m_sde_karras, CFG Scale: 7, Seed: 652753, Size: 1280x1920, Model hash: a3cd261baa, Model: pandorasBoxNSFW_v1PussBoots, Version: ComfyUI
耗时6.564253807067871秒
原图片021.jpg解析信息失败,使用默认提示词
生成图片的信息是：
Negative prompt: 
Steps: 35, Sampler: dpmpp_2m_sde_karras, CFG Scale: 7, Seed: 652753, Size: 1280x1920, Model hash: a3cd261baa, Model: pandorasBoxNSFW_v1PussBoots, Version: ComfyUI
耗时6.1394944190979秒
原图片024.jpg解析信息失败,使用默认提示词
生成图片的信息是：
Negative prompt: 
S

KeyboardInterrupt: 